In [ ]:
# @title Setup Environment.
from pathlib import Path
import os

lang, branch = 'en', 'main'
scripts_dir = Path.home() / 'ANXETY' / 'scripts'
out = scripts_dir / 'setup.py'

os.makedirs(out.parent, exist_ok=True)
!curl -sLo {out} https://raw.githubusercontent.com/remphanstar/LightningSdaigen/{branch}/scripts/setup.py

%run $out --lang=$lang --branch=$branch

# Update scripts_dir to use the path that setup.py determined
scripts_dir = Path(os.environ['scr_path']) / 'scripts'

---
## 1. Widgets 🔽

In [ ]:
# Model selection, vae, control-net and more.
%run $scripts_dir/$lang/widgets-{lang}.py

## 2. Downloading 🔽

In [ ]:
# Downloading libraries, repos, models and more.
%run $scripts_dir/$lang/downloading-{lang}.py

## 3. Start 🔽

In [ ]:
# Launch WebUI.
%run $scripts_dir/launch.py

In [ ]:
#
#
# FINAL CORRECTED CELL: This cell handles all setup and launch steps.
# It installs a clean environment, fixes all dependencies, and starts the WebUI.
#
#

import os
import sys
import subprocess
from pathlib import Path

# --- 1. Ensure System Tools Are Installed ---
print("💿 Installing system tools (aria2, lz4, pv)...")
!apt-get -y update > /dev/null 2>&1
!apt-get -y install aria2 lz4 pv > /dev/null 2>&1
print("✅ System tools are ready.")

# --- 2. Download and Extract the Correct Virtual Environment ---
HOME = Path('/content')
VENV = HOME / "venv"

if VENV.exists():
    print("Removing old virtual environment...")
    !rm -rf {VENV}

print("Downloading the corrected virtual environment...")
# Using the original, reliable venv link
venv_url = "https://huggingface.co/NagisaNao/ANXETY/resolve/main/python31018-venv-torch260-cu124-C-fca.tar.lz4"
archive_path = HOME / "venv.tar.lz4"
!aria2c -x 16 -s 16 -k 1M --console-log-level=error -c -d {HOME} -o {archive_path.name} "{venv_url}"

print("Extracting virtual environment...")
!pv {archive_path} | lz4 -d | tar xf - -C {HOME}
!rm {archive_path}
print("✅ Virtual environment is ready.")

# --- 3. Download and Set Up the WebUI ---
UI = "A1111" # You can change this to 'Forge' etc. if needed
WEBUI_PATH = HOME / UI
if not WEBUI_PATH.exists():
    print(f"Downloading {UI} WebUI...")
    webui_url = f"https://huggingface.co/NagisaNao/ANXETY/resolve/main/{UI}.zip"
    webui_zip = HOME / f"{UI}.zip"
    !aria2c -x 16 -s 16 -k 1M --console-log-level=error -c -d {HOME} -o {webui_zip.name} "{webui_url}"
    !unzip -q -o {webui_zip} -d {WEBUI_PATH}
    !rm {webui_zip}
    print(f"✅ {UI} WebUI is ready.")

# --- 4. Fix All Extension Dependencies ---
python_executable = VENV / 'bin' / 'python'
extensions_path = WEBUI_PATH / 'extensions'

print("Fixing all extension dependencies...")
# Force install all the correct, compatible library versions
!{python_executable} -m pip install --force-reinstall accelerate==0.21.0 onnx==1.14.0 onnxruntime==1.15.0 insightface==0.7.3 opencv-python==4.7.0.72 rembg supervision ifnude
# Clone the missing dependency for the Replacer extension
if not (extensions_path / 'sd-webui-segment-anything').exists():
    !git clone https://github.com/continue-revolution/sd-webui-segment-anything.git {extensions_path}/sd-webui-segment-anything
print("✅ All dependencies have been corrected.")


# --- 5. Launch the WebUI ---
print("\n🚀 Launching WebUI...")
%run {WEBUI_PATH}/launch.py --xformers --no-half-vae --enable-insecure-extension-access --disable-console-progressbars --theme dark

---
# Utilities